In [9]:
!pip install foliIum

  ERROR: Could not find a version that satisfies the requirement foliIum (from versions: none)
ERROR: No matching distribution found for foliIum


##scrap a table showing the list of canadian postal codes using Beautiful Soup
!pip install Beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install request
import requests
from bs4 import BeautifulSoup
import pandas as pd


##download url data from the internet
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
canada_data=BeautifulSoup(source,'lxml')

##Convert the table of postal codes to a dataframe
headline=['Postalcode','Borough','Neighborhood']
canadacodes=pd.DataFrame(columns=headline)

##loop to extract data

cell_data=canada_data.find('div', class_='mw-parser-output')
table=cell_data.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    canadacodes = canadacodes.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

##cleanup

canadacodes=canadacodes[canadacodes.Borough!='Not assigned']
canadacodes=canadacodes[canadacodes.Borough!=0]
canadacodes.reset_index(drop=True,inplace=True)

i=0

for i in range(0,canadacodes.shape[0]):
    if canadacodes.iloc[i][2] == 'Not assigned':
        canadacodes.iloc[i][2] = canadacodes.iloc[i][1]
        i = i+1
        
df = canadacodes.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

#data cleaning
#Drop "None" rows in DataFrame
#Drop any row which contains 'Not assigned' value
#All "Not assigned" will be replace to 'NaN' using numpy for convenience.
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]


In [13]:
install -c conda-forge folium

SyntaxError: invalid syntax (<ipython-input-13-2fde174b6893>, line 1)

In [10]:
# importing necessary libraries
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
##import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')

Libraries imported.


In [48]:
df2.tail()

,Postalcode,Borough,Neighborhood
175,M9V\n,Etobicoke\n,South Steeles / Silverstone / Humbergate / Jam...
176,M9W\n,Etobicoke\n,Northwest
177,M9X\n,Not assigned\n,
178,M9Y\n,Not assigned\n,
179,M9Z\n,Not assigned\n,


In [11]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

lst

[['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park / Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor / Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park / Ontario Provincial Government"],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Malvern / Rouge'],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill / Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M9B',
  'Etobicoke',
  'West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale'],
 ['M1C', 'Scarborough', 'Rouge Hill / Port Union / Highland Creek'],
 ['M3C', 'North York', 'Don Mills'],
 ['M4C', 'East York', 'Woodbine Heights'],
 ['M5C', 'Downtown Toronto', 'St. James Town'],
 ['M6C', 'York', 'Humewood-Cedarvale'],
 ['M9C',
  'Etobicoke',
  'Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood'],
 ['M1

In [16]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df_toronto = pd.DataFrame(lst, columns=cols)
print(df_toronto.shape)
# df[df.duplicated(['PostalCode'], keep=False)] - this would have shown the duplicate PostalCodes
df_toronto = df_toronto.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

(103, 3)


In [ ]:
df_toronto.to_csv('Toronto1_df.csv',index=False)


In [19]:
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [ ]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')
print(df2.shape)
df.tail()

In [52]:
##!pip install geocoder
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postcode))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

KeyboardInterrupt: 

In [20]:
toronto_geocsv = 'https://cocl.us/Geospatial_data'
!wget -q -O 'toronto_m.geospatial_data.csv' toronto_geocsv
geocsv_data = pd.read_csv(toronto_geocsv).set_index("Postal Code")
geocsv_data.head()


,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [23]:
df_toronto.to_csv('Toronto1_df.csv',index=False)

Toronto1_df = "Toronto1_df.csv"
TorontoPostalCodes = pd.read_csv(Toronto1_df).set_index("PostalCode")
TorontoPostalCodes.rename_axis("Postal Code", axis='index', inplace=True)
TorontoPostalCodes.head()


,Borough,Neighborhood
Postal Code,,
M1B,Scarborough,Malvern / Rouge
M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
M1E,Scarborough,Guildwood / Morningside / West Hill
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [24]:
toronto_neighborhoods = TorontoPostalCodes.join(geocsv_data)
toronto_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [25]:
toronto_neighborhoods

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029
M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577
M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476
